In [2]:
""" 
    Anirudh Sathish , CS20B1125 
    Question No 6 
"""

' \n    Anirudh Sathish , CS20B1125 \n    Question No 6 \n'

### Task 
- Consider the 128- dimensional feature vectors given in the “face feature vectors.csv” file.
- Use the following information to design and implement a Bayes Classifier.
#### Dataset Specifications:
- Total number of samples = 800
- Number of classes = 2 ( labelled as “male” and “female”)
- Samples from “1 to 400” belongs to class “male”
- Samples from “401 to 800” belongs to class “female”
- Number of samples per class = 400
#### Use the following information to design classifier:
- Number of test samples ( first 5 in each class) = 5
- Number of training samples ( remaining 395 in each class) = 395
- Number of dimensions = 128

In [3]:
# libraries 
import numpy as np 
import pandas as pd

/home/anirudh/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [4]:
# load data 
face = pd.read_csv("face.csv")
face.head()

,Unnamed: 0,Unnamed: 1,0,1,2,3,4,5,6,7,...,118,119,120,121,122,123,124,125,126,127
0,1,male,-0.066420,0.151611,0.027740,0.052771,-0.066105,-0.041232,-0.002637,-0.158467,...,0.025989,-0.001087,0.027260,-0.046754,-0.118619,-0.163774,-0.000590,-0.076400,0.107497,0.001567
1,2,male,-0.030614,0.049667,0.008084,-0.050324,0.007649,-0.063818,-0.019530,-0.119905,...,0.044229,-0.023900,-0.028108,0.040618,-0.146579,-0.141244,0.016162,0.017638,0.080610,-0.015930
2,3,male,-0.096178,0.061127,0.035326,-0.035388,-0.090728,-0.018634,-0.024315,-0.139786,...,0.111141,0.059436,-0.029222,0.042115,-0.222173,-0.116908,0.093428,0.017391,0.057652,0.086116
3,4,male,-0.103057,0.085044,0.078333,-0.035873,-0.028163,0.004924,0.007829,-0.017016,...,0.100793,-0.002644,-0.023388,0.029497,-0.139830,-0.119243,0.005306,-0.015100,0.161575,0.062462
4,5,male,-0.125815,0.120046,0.023131,-0.042901,0.038215,-0.049677,-0.054258,-0.130758,...,0.090197,0.067527,0.039926,0.047469,-0.056852,-0.076700,0.004966,0.028171,0.026041,0.084135


In [5]:
# split data into test

testData = pd.concat([face.iloc[:5],face.iloc[400:405]],axis = 0)
trainData = pd.concat([face.iloc[5:400],face.iloc[405:800]],axis = 0)


In [6]:
trainData.iloc[:,1]

5        male
6        male
7        male
8        male
9        male
        ...  
795    female
796    female
797    female
798    female
799    female
Name: Unnamed: 1, Length: 790, dtype: object

In [7]:
gender_classes = set(trainData.iloc[:,1])
classes = list(gender_classes)
print(classes)

#Applying the same indexing to train ,test
trainData.iloc[:,1] = trainData.iloc[:,1].apply(classes.index)
testData.iloc[:,1] = testData.iloc[:,1].apply(classes.index)

['female', 'male']


/tmp/ipykernel_87785/3618446032.py:6: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  trainData.iloc[:,1] = trainData.iloc[:,1].apply(classes.index)
/tmp/ipykernel_87785/3618446032.py:7: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  testData.iloc[:,1] = testData.iloc[:,1].apply(classes.index)


In [8]:
# lets calculate the covariance matrix and mean for the train data

trainingMean_class1 = trainData.iloc[5:400,2:128].mean().to_numpy()
trainingCov_class1 = np.cov(trainData.iloc[5:400,2:128].to_numpy().T)

trainingMean_class2 = trainData.iloc[405:800,2:128].mean().to_numpy()
trainingCov_class2 = np.cov(trainData.iloc[405:800,2:128].to_numpy().T)

trainingMean = [trainingMean_class1,trainingMean_class2]
trainingCov = [trainingCov_class1,trainingCov_class2]

# also find inverse of Covs 
trainingInvCov = [np.linalg.inv(x) for x in trainingCov]

In [9]:
def posteriorProbability(dim,covM,X,mean,invCovM):
    X = X.T
    #ignore denominator 
    #denominator = ((2*np.pi)**(dim/2))*((np.linalg.det(covM))**(1/2))
    power = (-1*(((X-mean).T @ invCovM) @ (X-mean))*(1/2))
    #print(power)
    numerator = np.exp(power)
    posteriorProb = numerator
    
    return posteriorProb

In [17]:
def classify(dim,covMs,X,means,invCovMs):

    # check across all classes 

    # maintain max var
    max = -1
    flag = -1  
    Maxs = []
    for i in range(0,2):
        Mean = np.resize(means[i],(means[i].shape[0],1))
        posteriorProb = posteriorProbability(dim,covMs[i],X,Mean,invCovMs[i])
        Maxs.append(posteriorProb)
        if posteriorProb > max: 
            max = posteriorProb
            flag = i
    
    # flag is the class
    if flag == 0:
        return 1 
    if flag == 1:
        return 0 

    return flag 

    

In [18]:
# running through test data to check for predictions 
dim = 128
predictions = []
for i in range(testData.shape[0]):
    X =  testData.iloc[i:i+1,2:128].to_numpy()
    resultant = classify(dim,trainingCov,X,trainingMean,trainingInvCov)
    predictions.append(resultant)

In [19]:
print(predictions)

[1, 1, 1, 1, 1, 1, 0, 0, 0, 0]


In [21]:
match = 0  
for i in range(len(testData)):
    expected = int(testData.iloc[i:i+1,1])
    prediction = predictions[i]
    if(prediction == expected):
        match+=1
accuracy = (match/len(testData))*100
print("Accuracy of the Bayesian Classifier :" ,accuracy)

Accuracy of the Bayesian Classifier : 90.0
